# Text generation with an RNN

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [4]:
import tensorflow as tf
import numpy as np
import os
import time

In [5]:
path = './niz.txt'

### Read the data

First, look in the text:

In [6]:
# Read, then decode for py2 compat.
text = open(path, 'rb').read().decode(encoding='unicode_escape')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 521681 characters


In [7]:
# Take a look at the first 250 characters in text
print(text[:250])


                  Zarathustra's Prologue

                            1.

  WHEN Zarathustra was thirty years old, he left his home and the lake
of his home, and went into the mountains. There he enjoyed his
spirit and his solitude, and for ten year


In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

74 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation.

The `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [9]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Now create the `tf.keras.layers.StringLookup` layer:

In [10]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

It converts from tokens to character IDs:

In [11]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[49, 50, 51, 52, 53, 54, 55], [72, 73, 74]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [12]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [13]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

You can `tf.strings.reduce_join` to join the characters back into strings.

In [14]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [15]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


In [16]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(521681,), dtype=int64, numpy=array([1, 2, 2, ..., 1, 1, 1])>

In [17]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [18]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

In [19]:
seq_length = 100


The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [20]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b'Z' b'a' b'r' b'a' b't' b'h' b'u' b's' b't'
 b'r' b'a' b"'" b's' b' ' b'P' b'r' b'o' b'l' b'o' b'g' b'u' b'e' b'\n'
 b'\n' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b'1' b'.' b'\n' b'\n' b' ' b' ' b'W' b'H' b'E' b'N' b' ' b'Z' b'a'
 b'r' b'a' b't' b'h' b'u' b's' b't' b'r' b'a' b' ' b'w' b'a' b's' b' '
 b't' b'h' b'i'], shape=(101,), dtype=string)


It's easier to see what this is doing if you join the tokens back into strings:

In [21]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b"\n                  Zarathustra's Prologue\n\n                            1.\n\n  WHEN Zarathustra was thi"
b'rty years old, he left his home and the lake\nof his home, and went into the mountains. There he enjoy'
b'ed his\nspirit and his solitude, and for ten years did not weary of it. But at\nlast his heart changed,'
b'- and rising one morning with the rosy dawn, he\nwent before the sun, and spake thus unto it:\n  Thou g'
b'reat star! What would be thy happiness if thou hadst not those\nfor whom thou shinest!\n  For ten years'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and
`label` are sequences. At each time step the input is the current character and the label is the next character.

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [22]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [23]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [24]:
dataset = sequences.map(split_input_target)

In [25]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"\n                  Zarathustra's Prologue\n\n                            1.\n\n  WHEN Zarathustra was th"
Target: b"                  Zarathustra's Prologue\n\n                            1.\n\n  WHEN Zarathustra was thi"


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [26]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

In [27]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [28]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [29]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 75) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [31]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  19200     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  76875     
                                                                 
Total params: 4,034,379
Trainable params: 4,034,379
Non-trainable params: 0
_________________________________________________________________


To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [32]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [33]:
sampled_indices

array([ 4,  7, 63, 15, 70, 35, 63, 56,  6,  1,  4, 73, 64, 17, 58, 20, 30,
       65, 54, 67, 64, 73, 20, 34, 26, 50, 32, 47, 65, 35, 42, 13, 42, 54,
       25, 67, 63, 70, 13, 65, 66, 57,  2, 46, 17, 43, 31, 29, 16, 74, 26,
       67, 18, 39, 70, 12, 18,  7, 10, 59,  4,  6, 51, 66, 40, 32, 37, 74,
       21, 67, 47, 41, 54, 17, 27, 34, 33, 49, 22, 62, 71,  2, 63,  5, 38,
       39, 57,  1, 52, 40, 66, 32, 74, 73, 52,  1, 42, 67, 50, 41])

Decode these to see the text predicted by this untrained model:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'reators and cultivators and sowers of the future;-\n  -Verily, not to a nobility which ye could purch'

Next Char Predictions:
 b'")o4vLoh(\n"yp6j9Gqfspy9KCbIYqLS2SfBsov2qri W6THF5zCs7Pv17).k"(crQINz:sYRf6DKJa;nw o\'OPi\ndQrIzyd\nSsbR'


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

### Attach an optimizer, and a loss function

The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [35]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 75)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.315539, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

74.85395

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

### Configure checkpoints

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [39]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

To keep training time reasonable, use 10 epochs to train the model. In Colab, set the runtime to GPU for faster training.

In [40]:
EPOCHS = 20

In [41]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
80/80 [==============================] - 18s 74ms/step - loss: 3.0869 - accuracy: 0.2137
Epoch 2/20
80/80 [==============================] - 6s 69ms/step - loss: 2.3304 - accuracy: 0.3407
Epoch 3/20
80/80 [==============================] - 6s 53ms/step - loss: 2.0647 - accuracy: 0.4034
Epoch 4/20
80/80 [==============================] - 5s 55ms/step - loss: 1.8517 - accuracy: 0.4601
Epoch 5/20
80/80 [==============================] - 5s 53ms/step - loss: 1.6922 - accuracy: 0.5019
Epoch 6/20
80/80 [==============================] - 5s 54ms/step - loss: 1.5749 - accuracy: 0.5318
Epoch 7/20
80/80 [==============================] - 5s 56ms/step - loss: 1.4856 - accuracy: 0.5542
Epoch 8/20
80/80 [==============================] - 6s 55ms/step - loss: 1.4153 - accuracy: 0.5728
Epoch 9/20
80/80 [==============================] - 5s 55ms/step - loss: 1.3538 - accuracy: 0.5887
Epoch 10/20
80/80 [==============================] - 5s 56ms/step - loss: 1.3017 - accuracy: 0.6020
Epoch 11

## Generate text

The following makes a single step prediction:

In [42]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [43]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [54]:
start = time.time()
states = None
next_char = tf.constant(['stars'])
result = [next_char]


for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)


result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

stars, and would fain
came, as from failures, laughing at
this gloomy throughous such a God, like the
scrobible, or affliction, and said:
  The shore the ten truths and exable himself.
  "But whatever happiness, unwilling to love and exulted:
  Everything injuredy and wickedest that he died too
young; one thing is mixnound themselves, like all
Uple?
  Ever virtue and a little cold, and his
seat for itself good and evil:- how wilt thou
remain all the flock of nothing service! So that
the night and the strangest thing, a physician: thy soul with leafter'ly about it up my clambera; verily, with worthy about, let

  ThE forest have I found the mourtain- oppresseth with great mouth. Ready to the
Name wishest not yet come,- what testity on more of my heart?
Well then, thou creath for thyself? As yet it still the country of my cave: thouragn hast thou
animals, thas good things; already not of ununglock. The dead, me for one and
there! Hust!
They call thee uptared of us power: so have perpease

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above.

In [55]:
start = time.time()
states = None
next_char = tf.constant(['stars', 'stars', 'stars', 'stars', 'stars'])
result = [next_char]

for n in range(500):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b'stars shall no one talk\nthus creative enamourity, those gods of springing leg,\nand their drunken, cannot midnight, and leisure it, many think imposed upon;\n  -In thee! Sumperied over day: so maketh them had not seek. Do not tender youn away!\n  And we went a frighten at his head wno lost them to the tarantly\nones! Since he was forsakenness pretence.\n  Gladly, thou thyself here as hath hanneth. To the incline in his mountain to me.\n  "We have loved his foremoth as with soor poverty.\n  Go out unto thee,'
 b'stars, however, whose\nmarriage-runger!\n  Smitten and jaughing, wings, thought out upon drien than\nany\nmore. Must now have your happiness shall he new\ntruth.- all grey laughter, place imperfect cripde;\nand this man idmost a\nbitter unto me, thou shright!-\n  -Then, ye might seek and become awar, that the\nlightest abso betimes, and wanteth to our fover round the\nhumanlest back; and, say slay all at once\ntheir limps of my type have the geturn and\nawfroyed, 

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [46]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [56]:
states = None
next_char = tf.constant(['love'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

love and of all egrained happy, all good and bad lords
in all new thoughts, and the fuller then only
do 
